In [1]:
from random import choice
import numpy as np
import regex as re
import pickle
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from colorama import Fore, Back, Style

In [528]:
'''CLF for x'''
with open('alak_retrained41768_clf.p', 'rb') as f: #alak_x15624_clf.p, alak4011_clf.p, alak_o26590_retrained_clf.p, alak_retrained41768_clf.p
    clf = pickle.load(f)

In [527]:
'''CLF for o'''
with open('alak_o_random16371a_clf.p', 'rb') as f: #alak_o_random16371_clf.p, alak_o_random16371a_clf.p, alak_retrained41768_clf.p
    clf_o = pickle.load(f)

In [526]:
'''CLF for kill x'''
with open('alak_o_kill12504_clf.p', 'rb') as f: # alak_o_kill12504_clf.p, alak_kill16490_clf.p
    clf_kill = pickle.load(f)

In [525]:
'''CLF for kill o'''
with open('alak_kill63051_clf.p', 'rb') as f: # alak_kill214732_clf.p, alak_kill63051_clf.p
    clf_kill_o = pickle.load(f)

In [568]:
class alak:
    '''
    Artificial Neural Network Alak Player:
    an artificial neural network (ANN) algorithm 
    to play the one-dimensional version of the ancient board game Go, or Alak
    '''
    def __init__(self, interactive=False, random=False, NN=True, show_mv=True, save_data=False, clf=clf, clf_kill=clf_kill, clf_o=clf_o, clf_kill_o=clf_kill_o):
        self.random = random
        self.interactive = interactive
        self.play_mode = 'interactive' if self.interactive else 'random'
        self.save_data = save_data
        self.nn = NN
        self.show_mv = show_mv
        self.clf = clf
        self.clf_kill = clf_kill
        self.clf_kill_o = clf_kill_o
        self.clf_o = clf_o
        self.board = 'xxxx__oooo'
        self.board_arr = []
        self.empty_board = []
        self.o_idx = []
        self.x_idx = []
        self.x = 'x'
        self.o = 'o'
        self.pattern_x = 'ox+o'
        self.pattern_o = 'xo+x'
        self.winner = ''
        self.player1 = ''
        self.player2 = ''
        self.move_data = []
        self.board_data = np.ndarray((0, 10))
        self.kill_board = []
        self.kill_move = []
        self.move_from = 0
        self.move_to = 0
        self.x_win = 0
        self.o_win = 0
        self.suicide = False
        self.round_count = 0
        self.move_count = 0
        self.kill_count = 0
        self.gain = 0
    
    def test_remove(self):
        """
        This function tests out special cased moves for both sides:
        - simple kill
        - double kill 
        - double kill that involves more than one piece 
        - suicide moves.
        """
        board_list = ['xoxoxx____', 'xooxooxx__', '__xoo__oxo', '_xoxxxoo____', '_x_oxxoo____',  '_xoxx___x_']
        board_expect = ['x_x_xx____', 'x__x__xx__', '__xoo__o_o', '_xo___oo____', '_x_o__oo____', '_x_xx___x_']
        off_side = ['x', 'x', 'x', 'o', 'o', 'o']
        fail_num = 0
        print("Testing test_remove after move from offensive side:")
        for i, board in enumerate(board_list):
            if self.show_mv: print("\nAfter {:s} move:".format(off_side[i]))
            self.board = board
            if self.show_mv:
                print("Test {:d}".format(i))
                print('Board: ', self.board)
            self.check_kill(off_side[i])
            try:
                assert self.board == board_expect[i]
            except:
                print('i', i)
                fail_num += 1
        tot_num = len(board_list)
        print('{:d} out of {:d} tests passed'.format(len(board_list) - fail_num, len(board_list)))
        if fail_num > 0: print(Fore.RED + '{:d} out of {:d} tests failed'.format(fail_num, len(board_list)))
            
    def move(self, player, p1, p2, board):
        """
        Move player from position 1 to position 2 then return altered board
        """
        if p1 > p2:
            p1, p2 = p2, p1
            sym1 = player
            sym2 = '_'
        else:
            sym1 = '_'
            sym2 = player
        board = board[0:p1] + sym1 + board[p1 + 1:p2] + sym2 + board[p2 + 1:]
        return board

    def get_board_arr(self):
        """
        Convert board string into board array
        """
        self.board_arr = np.array(list(self.board))

    def find(self):
        """
        Find the index of players and empty board
        """
        self.find_xo()
        self.find_empty()

    def find_empty(self):
        """
        Find the index of empty board
        """
        self.empty_board = np.where(self.board_arr == '_')[0]

    def find_xo(self):
        """
        Find the index of players
        """
        self.get_board_arr()
        self.x_idx = np.where(self.board_arr == 'x')[0]
        self.o_idx = np.where(self.board_arr == 'o')[0]

    def print_board(self):
        """
        Print current board
        """
        if self.show_mv: print('{}\n{}'.format(self.board, '0123456789'))
    
    def save_board(self, board):
        """
        Append 1 for x, -1 for o, and 0 for empty
        """
        board_status = []
        for i in board:
            if i == 'x':
                board_status.append(1)
            elif i == 'o':
                board_status.append(-1)
            else:
                board_status.append(0)
        return board_status

    def store_move_data(self, move_count):
        """
        Append 1 if x won, 0 if o
        """
        for move in range(move_count):
            if self.winner == 'x':
                self.move_data.append(1)
            elif self.winner == 'o':
                self.move_data.append(0)

    def player_turn(self, player, player_idx):
        """
        Call move player and check kill
        """
        self.move_player(player, player_idx)
        self.check_kill(player)
        
    def get_match(self, pattern, board):
        """
        Return true if there is match
        """
        return re.search(pattern, board)
    
    def remove(self, pattern):
        """
        Remove matching pattern during kill
        """
        self.get_board_arr()
        search = re.search(pattern, self.board)

        for i in range(search.start(0) + 1, search.end(0) - 1):
            self.board_arr[i] = '_'
            
        if self.show_mv:
            if (search.start(0) + 1) == (search.end(0) - 2):
                print('\n{} at {} is captured'.format(search.group(0)[1], search.start(0) + 1))
            else:
                print('\n{} from {} to {} are captured'.format(search.group(0)[1], search.start(0) + 1,
                                                               search.end(0) - 2))
        return self.board_arr

    def kill(self, pattern):
        """
        Check if there is a match, call remove function
        """
        kill = False
        while re.search(pattern, self.board) != None:
            kill = True
            self.kill_board.append(self.save_board(self.board))
            self.kill_count += 1
            self.board_arr = self.remove(pattern)
            self.board = ''.join([str(elem) for elem in self.board_arr])
            self.print_board()
        return kill
    
    def check_kill(self, player):
        """
        Check if it's a kill or suicide
        """
        if player == self.o:
            if self.kill(self.pattern_x):
                self.gain = 1
                if self.show_mv: print(Fore.BLUE + "Nice move o!" + Style.RESET_ALL)
                for i in range(self.kill_count):
                    self.kill_move.append(0)
                self.kill_count = 0
            if self.kill(self.pattern_o):
                self.suicide = True
                self.gain = -1
                if self.show_mv: print(Back.RED + "SUICIDE MOVE!" + Style.RESET_ALL)
                self.kill_board.pop()
                self.kill_count -= 1
        elif player == self.x:
            if self.kill(self.pattern_o):
                self.gain = 1
                if self.show_mv: print(Fore.BLUE + "Nice move x!" + Style.RESET_ALL)
                for i in range(self.kill_count):
                    self.kill_move.append(1)
                self.kill_count = 0
            if self.kill(self.pattern_x):
                self.suicide = True
                self.gain = -1
                if self.show_mv: print(Back.RED + "SUICIDE MOVE!" + Style.RESET_ALL)
                self.kill_board.pop()
                self.kill_count -= 1
        if self.show_mv: print(Fore.GREEN + 'gain:{}'.format(self.gain) + Style.RESET_ALL)
        self.gain = 0
        
    def move_player(self, player, player_idx):
        """
        Move random player from position1 to position2 then print board
        """
        position1 = choice(player_idx)
        position2 = choice(self.empty_board)
        self.board = self.move(player, position1, position2, self.board)
        if self.show_mv: print('\n{} moved from {} to {}'.format(player, position1, position2))
        self.print_board()
        
    def move_user(self, user):
        """
        Move user player from input move from to input move to, update and print board
        """
        self.board = self.move(user, self.move_from, self.move_to, self.board)
        self.check_kill(user)
        self.print_board()
    
    def move_comp(self, comp):
        """
        Move computer if NN is true call predict proba else move player turn randomly
        """
        if self.show_mv: print('\n== {} TURN =='.format(comp))
        if self.nn:
            self.player_predict_proba(self.x, self.x_idx) if comp == 'x' else self.player_predict_proba(self.o, self.o_idx)
        else:
            self.player_turn(self.x, self.x_idx) if comp == 'x' else self.player_turn(self.o, self.o_idx)
    
    def player_predict_proba(self, player, player_idx):
        """
        Move player based on NN prediction then call check kill
        """
        self.predict_proba_kill(player, player_idx)
        self.check_kill(player)
    
    def predict_prob(self, player, player_idx):
        """
        Predict probability to get the best move
        """
        max_prob, position1, position2 = 0, 0, 0
        empty = self.empty_board
        target = 1 if player == self.x else 0
        pattern = self.pattern_x if player == self.x else self.pattern_o
        pattern2 = self.pattern_o if player == self.x else self.pattern_x
        clf = self.clf if player == self.x else self.clf_o
        breaker = False
       
        for i in player_idx:
            if breaker: break
            for j in empty:
                temp_move = self.move(player, i, j, self.board)
                if self.get_match(pattern, temp_move):
                    if not self.get_match(pattern2, temp_move):
                        continue
                    else:
                        breaker, position1, position2 = True, i, j
                        break
                temp = np.atleast_2d(self.save_board(temp_move))
                predicted_prob = self.clf.predict_proba(temp)
                if predicted_prob[0][target] > max_prob:
                    max_prob = predicted_prob[0][target]
                    position1, postion2 = i, j
        return position1, position2
    
    def predict_proba_kill(self, player, player_idx):
        """
        Predict probability and kill probability to get the best move
        """
        max_prob, max_prob_kill, position1, position2 = 0, 0, 0, 0
        empty = self.empty_board
        target = 1 if player == self.x else 0
        pattern = self.pattern_x if player == self.x else self.pattern_o
        pattern2 = self.pattern_o if player == self.x else self.pattern_x
        clf = self.clf if player == self.x else self.clf_o
        clf_kill = self.clf_kill if player == self.x else self.clf_kill_o
        breaker = False

        for i in player_idx:
            if breaker: break
            for j in empty:
                temp_move = self.move(player, i, j, self.board)
                if self.get_match(pattern, temp_move):
                    if not self.get_match(pattern2, temp_move):
                        continue
                    else:
                        breaker, position1, position2 = True, i, j
                        break
                temp = np.atleast_2d(self.save_board(temp_move))
                predicted_prob = clf.predict_proba(temp)
                predicted_kill = clf_kill.predict_proba(temp)
                if predicted_prob[0][target] > max_prob and predicted_kill[0][target] > max_prob_kill:
                    max_prob = predicted_prob[0][target]
                    max_prob_kill = predicted_kill[0][target]
                    position1, position2 = i, j
        if position1 == position2:
            position1, position2 = self.predict_prob(player, player_idx)
        if self.show_mv: print('\n{} moved from {} to {}'.format(player, position1, position2))
        self.board = self.move(player, position1, position2, self.board)
        self.print_board()

    def check_vals(self):
        """
        Check length of x and o indices
        """
        return len(self.o_idx) <= 1 or len(self.x_idx) <= 1

    def get_winner(self):
        """
        Determine winner
        """
        if len(self.o_idx) <= 1:
            self.winner = self.x
            self.x_win += 1
        elif len(self.x_idx) <= 1:
            self.winner = self.o
            self.o_win += 1

    def after_win(self):
        """
        Call get winner, store move data and print winner
        """
        self.get_winner()
        self.store_move_data(len(self.board_data))
        if self.show_mv: print(Back.YELLOW + '\nGame over, player {} wins!'.format(self.winner) + Style.RESET_ALL)

    def reset_game(self):
        """
        Reset game
        """
        self.board = 'xxxx__oooo'
        self.board_arr = []
        self.board_data = np.ndarray((0, 10))
        self.empty_board = []
        self.o_idx = []
        self.x_idx = []
        self.suicide = False
        self.round_count = 0
        self.move_count = 0

    def store_board_data(self):
        """
        Store board data
        """
        temp_board = np.array(self.save_board(self.board))
        self.board_data = np.vstack((self.board_data, temp_board))

    def play_round(self, max_round):
        """
        Play random game
        """
        self.print_board()
        for r in range(max_round):
            self.round_count += 1
            if self.save_data:
                if self.round_count >= 15: break
            self.find()
            if self.show_mv:
                print('\nROUND: {}\n============================='.format(self.round_count))
                print('X: {}\nO: {}'.format(len(self.x_idx), len(self.o_idx)))
            
            # ADD NOISE
            # if self.round_count == 2 or self.round_count == 4:
                # self.player_turn(self.o, self.o_idx)
            # else:
                # self.player_turn(self.o, self.o_idx) if self.nn == False else self.player_predict_proba(self.o, self.o_idx)            
            
            # if self.round_count == 2 or self.round_count == 4:
                # self.player_turn(self.x, self.x_idx) if self.nn == False else self.player_predict_proba(self.x, self.x_idx)
            # else:
                # self.player_turn(self.x, self.x_idx)
        
            # self.player_turn(self.x, self.x_idx)
            # self.player_turn(self.o, self.o_idx)
            # self.player_turn(self.o, self.o_idx) if self.nn == False else self.player_predict_proba(self.o, self.o_idx)
            
            self.player_turn(self.x, self.x_idx) if self.nn == False else self.player_predict_proba(self.x, self.x_idx)
            self.store_board_data()
            self.move_count += 1            
            if self.save_data:
                if self.suicide: break
            self.find_xo()
            if self.check_vals():
                self.after_win()
                break

            self.find()
            
            # ADD NOISE
            # if self.round_count == 2 or self.round_count == 4:
                # self.player_turn(self.x, self.x_idx) if self.nn == False else self.player_predict_proba(self.x, self.x_idx)
            # else:
                # self.player_turn(self.x, self.x_idx)
            
            # if self.round_count == 2 or self.round_count == 4:
                # self.player_turn(self.o, self.o_idx)
            # else:
                # self.player_turn(self.o, self.o_idx) if self.nn == False else self.player_predict_proba(self.o, self.o_idx)

            # self.player_turn(self.o, self.o_idx) if self.nn == False else self.player_predict_proba(self.o, self.o_idx)
            # self.player_turn(self.x, self.x_idx) if self.nn == False else self.player_predict_proba(self.x, self.x_idx)
            # self.player_turn(self.x, self.x_idx)
         
            self.player_turn(self.o, self.o_idx)
            self.store_board_data()
            self.move_count += 1
            if self.save_data:
                if self.suicide: break
            self.find_xo()
            if self.check_vals():
                self.after_win()
                break
                
    def get_input(self, user):
        """
        Call get input for move from and move to and display move
        """
        if self.show_mv: print('== {} TURN ==\n'.format(user))
        self.move_from = self.get_move_from(user, 'Enter move from: ')
        self.move_to = self.get_move_to('Enter move to: ')
        if self.show_mv: print('{} moved from {} to {}'.format(user, self.move_from, self.move_to))
    
    def get_user_input(self, condition1, condition2, prompt):
        """
        Validate input
        """
        while True:
            try:
                answer = input(prompt)
                if answer == condition1 or answer == condition2:
                    break
                else:
                    print('Enter "{}" or "{}", try again.'.format(condition1, condition2))
            except Exception as e:
                print('Invalid entry.')
        return answer
    
    def get_move_from(self, user, prompt):
        """
        Validate input for position1
        """
        user_idx = np.where(self.board_arr == user)[0]
        while True:
            try:
                position = int(input(prompt))
                if position in user_idx:
                    break
                else:
                    print('Try again, please select from: {}'.format(user_idx))
            except Exception as e:
                print('Invalid position.')
        return position
    
    def get_move_to(self, prompt):
        """
        Validate input for position2
        """
        self.find_empty()
        while True:
            try:
                position = int(input(prompt))
                if position in self.empty_board: 
                    break
                else:
                    print('Try again, please select from: {}'.format(self.empty_board))
            except Exception as e:
                print('Invalid position.')
        return position
    
    def move_interactive(self, player, mode):
        """
        Move players in interactive game
        """
        if mode == '2': 
            self.get_input(player)
            self.move_user(player)
        else :
            if player == self.player1:
                self.get_input(player)
                self.move_user(player)
            else:
                self.move_comp(player)
        self.store_board_data()
        self.move_count += 1
        self.find_xo()
        
    def play_interactive(self, max_round):
        '''
        Play an interactive game
        '''
        game_mode = self.get_user_input('1', '2', 'Enter "1" to play with Alak or "2" for two players: ')
        self.player1 = self.get_user_input(self.x, self.o, '\nChoose stones, enter "x" or "o": ')
        self.player2 = self.x if self.player1 == 'o' else self.o
        go_first = self.get_user_input('y', 'n', 'Do you want to go first? (y/n) ')
        
        self.print_board()
        for r in range(max_round):
            self.round_count += 1
            self.find()
            if self.show_mv:
                print('\nROUND: {}\n============================='.format(self.round_count))
                print('X: {}\nO: {}'.format(len(self.x_idx), len(self.o_idx)))
            
            if go_first == 'y':
                self.move_interactive(self.player1, game_mode)
                if self.check_vals():
                    self.after_win()
                    break      
                self.find()
                self.move_interactive(self.player2, game_mode)
                if self.check_vals():
                    self.after_win()
                    break
            else:
                self.move_interactive(self.player2, game_mode)
                if self.check_vals():
                    self.after_win()
                    break
                self.move_interactive(self.player1, game_mode)
                if self.check_vals():
                    self.after_win()
                    break 
                    
    def play_games(self, n_game):
        '''
        Play games with option of interactive or random mode
        '''
        max_round = 100
        data = np.ndarray((0, 10))
        print(Fore.BLUE +'\n===========ALAK GO!==========\n' + Style.RESET_ALL)
        for game in range(n_game):
            if self.play_mode == 'interactive':
                self.play_interactive(max_round)
            elif self.play_mode == 'random':
                self.play_round(max_round)
            if not self.save_data:
                data = np.vstack((data, self.board_data))
            else:
                # if self.round_count < 15:
                if self.round_count < 15 and self.suicide == False:
                    data = np.vstack((data, self.board_data))
            self.reset_game()
        print(Fore.BLUE + 'x winning rate: {:.2f}%'.format(self.x_win / n_game * 100))
        print('o winning rate: {:.2f}%'.format(self.o_win / n_game * 100) + Style.RESET_ALL)
        return self.kill_move, np.array(self.kill_board)
        # return self.move_data, data

In [570]:
'''Main: NN o vs. Random x, Show Move(show_mv) False'''
a = alak(interactive=False, random=True, NN=True, show_mv=False, save_data=False)
n_game = 100
move_data, board_data = a.play_games(n_game)


===========ALAK GO!==========

x winning rate: 0.00%
o winning rate: 100.00%


In [567]:
'''Main: NN x vs. Random o, Show Move(show_mv) True'''
a2 = alak(interactive=False, random=True, NN=True, show_mv=True, save_data=False)
n_game = 10
move_data, board_data = a2.play_games(n_game) 


===========ALAK GO!==========

xxxx__oooo
0123456789

ROUND: 1
X: 4
O: 4

x moved from 1 to 5
x_xx_xoooo
0123456789
gain:0

o moved from 9 to 4
x_xxoxooo_
0123456789

x at 5 is captured
x_xxo_ooo_
0123456789
Nice move o!
gain:1

ROUND: 2
X: 3
O: 4

x moved from 0 to 5
__xxoxooo_
0123456789

o at 4 is captured
__xx_xooo_
0123456789
Nice move x!
gain:1

o moved from 7 to 0
o_xx_xo_o_
0123456789
gain:0

ROUND: 3
X: 3
O: 3

x moved from 2 to 7
o__x_xoxo_
0123456789

o at 6 is captured
o__x_x_xo_
0123456789
Nice move x!
gain:1

o moved from 8 to 4
o__xox_x__
0123456789

o at 4 is captured
o__x_x_x__
0123456789
SUICIDE MOVE!
gain:-1

Game over, player x wins!
xxxx__oooo
0123456789

ROUND: 1
X: 4
O: 4

x moved from 1 to 5
x_xx_xoooo
0123456789
gain:0

o moved from 9 to 4
x_xxoxooo_
0123456789

x at 5 is captured
x_xxo_ooo_
0123456789
Nice move o!
gain:1

ROUND: 2
X: 3
O: 4

x moved from 0 to 5
__xxoxooo_
0123456789

o at 4 is captured
__xx_xooo_
0123456789
Nice move x!
gain:1

o moved from 8

In [541]:
'''Main Interactive Show Move True'''
a3 = alak(interactive=True, random=False, NN=True, show_mv=True, save_data=False)
move_data, board_data = a3.play_games(1) 


===========ALAK GO!==========



Enter "1" to play with Alak or "2" for two players:  1

Choose stones, enter "x" or "o":  o
Do you want to go first? (y/n)  n


xxxx__oooo
0123456789

ROUND: 1
X: 4
O: 4

== x TURN ==

x moved from 1 to 5
x_xx_xoooo
0123456789
gain:0
== o TURN ==



Enter move from:  7
Enter move to:  4


o moved from 7 to 4

x at 5 is captured
x_xxo_o_oo
0123456789
Nice move o!
gain:1
x_xxo_o_oo
0123456789

ROUND: 2
X: 3
O: 4

== x TURN ==

x moved from 0 to 5
__xxoxo_oo
0123456789

o at 4 is captured
__xx_xo_oo
0123456789
Nice move x!
gain:1
== o TURN ==



Enter move from:  8
Enter move to:  4


o moved from 8 to 4

x at 5 is captured
__xxo_o__o
0123456789
Nice move o!
gain:1
__xxo_o__o
0123456789

ROUND: 3
X: 2
O: 3

== x TURN ==

x moved from 2 to 5
___xoxo__o
0123456789

o at 4 is captured
___x_xo__o
0123456789
Nice move x!
gain:1
== o TURN ==



Enter move from:  9
Enter move to:  4


o moved from 9 to 4

x at 5 is captured
___xo_o___
0123456789
Nice move o!
gain:1
___xo_o___
0123456789

Game over, player o wins!
x winning rate: 0.00%
o winning rate: 100.00%


In [572]:
'''Check length of games data'''
print(len(move_data))
print(len(board_data))
print(move_data)
print(board_data)

215
215
[0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[[ 1 -1  1 ... -1  0 -1]
 [ 1  1  0 ... -1  0 -1]
 [ 1  1  0 ...  0  0 -1]
 ...
 [ 0  1  1 ... -1  0 -1]
 [ 0 -1  1 ...  1  0 -1]
 [ 1 -1  1 ... -1  0 -1]]


In [517]:
'''Save Games data'''
with open('alak_retrained176908.p', 'wb') as f:
    pickle.dump((move_data, board_data), f)

In [548]:
'''Open games data'''
with open('alak_retrained176908.p', 'rb') as f: #alak_o_kill12504.p
    train_data = pickle.load(f)
y, X = train_data
print(len(X))
print(len(y))

176908
176908


In [ ]:
'''Grid Search'''

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.5)

param_grid = {'hidden_layer_sizes': [(100, 30, 20), (250,), (100,), (40, 4)],
              'learning_rate_init': [.001, .005, .001, .05, .01, .0001],
              'random_state': [0, 2, 5, 32],
             }
clf4 = GridSearchCV(
    MLPClassifier(solver='lbfgs', activation = 'tanh', alpha=1e-10, max_iter=100000), param_grid
)
clf4 = clf4.fit(X_train, y_train)

print('Training Score:', clf4.score(X_train, y_train))

In [18]:
'''Get validation score'''
predicted = clf4.predict(X_test)
print('Prediction Score:', clf4.score(X_test, y_test))
print("\nFor the classifier, {}".format(clf4, metrics.classification_report(y_test, predicted)))

print("this is the classification report:\n{}\n".format(metrics.classification_report(y_test, predicted)))
print("Confusion matrix:\n{}\n".format(metrics.confusion_matrix(y_test, predicted)))

Prediction Score: 0.9706850134609633

For the classifier, GridSearchCV(estimator=MLPClassifier(activation='tanh', alpha=1e-10,
                                     max_iter=100000, solver='lbfgs'),
             param_grid={'hidden_layer_sizes': [(100, 30, 20), (250,), (100,),
                                                (40, 4)],
                         'learning_rate_init': [0.001, 0.005, 0.001, 0.05, 0.01,
                                                0.0001],
                         'random_state': [0, 2, 5, 32]})
this is the classification report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2120
           1       0.98      0.94      0.96      1223

    accuracy                           0.97      3343
   macro avg       0.97      0.96      0.97      3343
weighted avg       0.97      0.97      0.97      3343


Confusion matrix:
[[2095   25]
 [  73 1150]]



In [106]:
'''Train CLF'''
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.5, random_state=2)

clf3 = MLPClassifier(solver='lbfgs', alpha=1e-10, hidden_layer_sizes=(40, 4), activation='tanh', \
                    random_state=2, max_iter=1000000, learning_rate_init=0.001)
clf3 = clf3.fit(X_train, y_train)
print('Training Score:', clf3.score(X_train, y_train))

Training Score: 0.97808701215611


In [468]:
'''Get validation score'''
predicted = clf3.predict(X_test)
print('Prediction Score:', clf3.score(X_test, y_test))
print("\nFor the classifier, {}".format(clf3, metrics.classification_report(y_test, predicted)))

print("this is the classification report:\n{}\n".format(metrics.classification_report(y_test, predicted)))
print("Confusion matrix:\n{}\n".format(metrics.confusion_matrix(y_test, predicted)))

Prediction Score: 0.9314786439379429

For the classifier, MLPClassifier(activation='tanh', alpha=1e-10, max_iter=1000000, random_state=5,
              solver='lbfgs')
this is the classification report:
              precision    recall  f1-score   support

           0       0.91      0.95      0.93      9853
           1       0.95      0.92      0.93     11031

    accuracy                           0.93     20884
   macro avg       0.93      0.93      0.93     20884
weighted avg       0.93      0.93      0.93     20884


Confusion matrix:
[[ 9312   541]
 [  890 10141]]



In [469]:
'''save clf'''
with open ('alak_retrained41768_clf.p', 'wb') as f:
    pickle.dump(clf3,f)

In [573]:
'''Test Legal Move'''
a2.test_remove()

Testing test_remove after move from offensive side:

After x move:
Test 0
Board:  xoxoxx____

o at 1 is captured
x_xoxx____
0123456789

o at 3 is captured
x_x_xx____
0123456789
Nice move x!
gain:1

After x move:
Test 1
Board:  xooxooxx__

o from 1 to 2 are captured
x__xooxx__
0123456789

o from 4 to 5 are captured
x__x__xx__
0123456789
Nice move x!
gain:1

After x move:
Test 2
Board:  __xoo__oxo

x at 8 is captured
__xoo__o_o
0123456789
SUICIDE MOVE!
gain:-1

After o move:
Test 3
Board:  _xoxxxoo____

x from 3 to 5 are captured
_xo___oo____
0123456789
Nice move o!
gain:1

After o move:
Test 4
Board:  _x_oxxoo____

x from 4 to 5 are captured
_x_o__oo____
0123456789
Nice move o!
gain:1

After o move:
Test 5
Board:  _xoxx___x_

o at 2 is captured
_x_xx___x_
0123456789
SUICIDE MOVE!
gain:-1
6 out of 6 tests passed
